A Raw use of Agents working, with text_generation

In [ ]:
from google.colab import userdata
token=userdata.get('HF_TOKEN')

In [ ]:
import os
from huggingface_hub import InferenceClient
os.environ["HF_TOKEN"]=token
client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct",token=token)

In [6]:
output = client.text_generation(
    "The capital of France is",
    max_new_tokens=100,
)
print(output)

 Paris. The capital of Italy is Rome. The capital of Spain is Madrid. The capital of Germany is Berlin. The capital of the United Kingdom is London. The capital of Australia is Canberra. The capital of China is Beijing. The capital of Japan is Tokyo. The capital of India is New Delhi. The capital of Brazil is Brasília. The capital of Russia is Moscow. The capital of South Africa is Pretoria. The capital of Egypt is Cairo. The capital of Turkey is Ankara. The


In [15]:
prompt="""<|begin_of_text|><|start_header_id|>user<|end_header_id|>
The capital of france is<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
output=client.text_generation(
    prompt,
    max_new_tokens=100
)
print(output)




The capital of France is Paris.


In [25]:
output=client.chat.completions.create(
    messages=[
        {"role":"user","content":"The captial of France is"},
    ],
    stream=False,
    max_tokens=100,
)
print(output.choices[0].message.content)

Paris.


A complex System prompt, that contains:
1. Information of the tools
2. Cycle instructions(Thought->Action->Observation)

Example below:




In [ ]:
SYSTEM_PROMPT="""Answer the following questions as best you can. You have access to the following tools:
get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json.
Specifically, this json should have an 'action' key (with the name of the tool to use) and an 'action_input' key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args:{"location":{"type":"String"}}
example use:

{{
  "action":"get_weather",
  "action_input":{"location":"New delhi"}
}}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:

Action:
$JSON_BLOB(inside markdown cell)

Observation: the result of the action. This Observation is unique, complete, and the source of trurth
...(this Thought/Action/Observation can repeat N times, you should take several steps  when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at time.)

You must always end your output with the following format:
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer."""

**Prompting using text_generation method**

In [ ]:
prompt=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{SYSTEM_PROMPT}
<|eot_id|><|start_header_id|>user<|end_header_id|>
What's the weather in Delhi?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

or the other way

In [ ]:
messages=[
    {"role":"system","content":SYSTEM_PROMPT},
    {"role":"user","content":"What's the weatther in Delhi"}
]

from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.apply_chat_template(messages,tokenize=False,add_generation_prompt=True)